# 引言

Python提供了一种特殊的写法，叫做推导(comprehension)，可以简洁地迭代列表、字典和集合等数据结构，并根据迭代结果生成另一套数据。

Python把这种理念也运用到了函数上面，产生了生成器(generator)，它可以让函数每次返回一系列值中的一个。凡是可以使用迭代器的任务都支持生成器函数。

# #27 用列表推导取代map与filter

假设我们要用列表中每个元素的平方值构建一份新的列表，传统的写法是采用`for`循环来写。

In [1]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squares = []
for x in a:
    squares.append(x**2)
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


这段代码可以改用列表推导来写。

In [3]:
squares = [x**2 for x in a]
squares

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

当然也可以用`map`实现该功能，它能从多个列表中分别取出当前位置上的元素，并把它们当作参数传递给映射函数，以求出新列表在这个位置上的元素值。但这看起来有点繁琐。

In [6]:
alt = map(lambda x: x ** 2 , a)

列表推导式还有一个地方比`map`好，就是它能方便地过滤原列表，把某些输入值对应的计算结果从输出结果中排除。

In [8]:
even_squares = [x**2 for x in a if x % 2 == 0]
even_squares

[4, 16, 36, 64, 100]

这种功能也可以通过内置的`filter`与`map`实现，但是这两个函数相结合的写法可读性没那么好。

In [9]:
alt = map(lambda x: x**2, filter(lambda x: x % 2 ==0, a))
assert even_squares == list(alt)

==字典与几何也有相应的推导机制，分别叫字典推导与集合推导。==

In [10]:
even_squares_dict = {x: x**2 for x in a if x % 2 == 0}
threes_cubed_set = {x**3 for x in a if x % 3 == 0}
print(even_squares_dict)
print(threes_cubed_set)

{2: 4, 4: 16, 6: 36, 8: 64, 10: 100}
{216, 729, 27}


如果改用`map`与`filter`实现，那么还必须调用相应的构造器，这会让代码变得很长。

In [11]:
alt_dict = dict(map(lambda x: (x, x**2),
				filter(lambda x: x % 2 == 0, a)))
alt_set = set(map(lambda x: x**3,
	          filter(lambda x: x % 3 == 0, a)))
assert even_squares_dict == alt_dict
assert threes_cubed_set == alt_set

# #28 控制推导逻辑的子表达式不要超过两个

除了最基本的用法外，列表推导式还支持多层循环。例如把矩阵转化成普通的一维列表，那么可以在推导时，使用两条`for`子表达式。这些子表达式会按照从左到右的顺序解读。

In [12]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row] #先是得到row，再遍历row得到x
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


多层循环还可以用来重制那种两层深的结构。例如，如果要根据二维矩阵里每个元素的平方值构建一个新的二维矩阵，那么可以采用下面的写法。

In [14]:
squared = [[x**2 for x in row] for row in matrix]
squared

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]

这看上去有点复杂，因为它把小的推导式逻辑`[x**2 for x in row]`嵌到了大的推导逻辑里面。大的推导逻辑用来决定新矩阵里的每一行，小的推导逻辑决定行中的每个元素。

这行语句总体上不难理解。但如果推导过程中还要再加一层循环，那么语句会很长，必须写成多行。比如下面把一个三维矩阵化成普通一维列表。

In [15]:
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
]
flat = [x for sublist1 in my_lists
        for sublist2 in sublist1
        for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


此时，采用列表推导式来实现，其实并不会比传统的`for`循环节省多少代码。下面用`for`循环来写一次，这要比刚才那种三层矩阵的列表推导式更加清晰。

In [17]:
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


推导的时候，可以使用多个`if`条件。如果这些`if`条件出现在同一层循环内，那么它们之间默认是`and`关系。例如，如果要用原列表中大于4且是偶数的值来构建新列表，那么既可以连用两个`if`，也可以只用一个`if`。

In [18]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 and x % 2 == 0]
print(b)
print(c)
assert b and c
assert b == c

[6, 8, 10]
[6, 8, 10]


在推导时，每一层的`for`子表达式都可以带有`if`条件。例如，要根据原矩阵构建新的矩阵，把其中各元素之和大于等于10的那些行选出来，而且只保留其中能被3整除的那些元素。
这个逻辑用列表推导来写，并不需要太多的代码，但是会很难理解。

In [19]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 ==0]
            for row in matrix if sum(row) >= 10]
filtered

[[6], [9]]

总之，在表示推导逻辑时，最多只应该写两个子表达式(两个`if`条件、两个`for`循环或一个`if`条件和一个`for`循环)。
只要实现的逻辑比这复杂，就应该采用普通的`if`与`for`来实现，且可以考虑编写辅助函数。

# #29 用赋值表达式消除推导中的重复代码

推导列表、字典与集合等变体结构时，经常要在多个地方用到同一个计算结果。例如，我们要给制作紧固件的公司编程以管理订单。顾客下单后，我们要判断当前的库存能否满足这份订单，即，要核查每种产品的数量有没有达到可以发货的最低限制(8个为一批，至少得有一批才能发货)。


In [20]:
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips'] # 库存中无 clip

def get_batches(count, size):
    return count // size

result = {}
for name in order:
    count = stock.get(name, 0)
    batches = get_batches(count, 8)
    if batches:
        result[name] = batches

print(result)

{'screws': 4, 'wingnuts': 1}


这段循环逻辑，如果改用字典推导来写，会简单一些。

In [21]:
found = {name: get_batches(stock.get(name,0), 8)
         for name in order
         if get_batches(stock.get(name,0), 8)}
found

{'screws': 4, 'wingnuts': 1}

这样写虽然简单，但是，`get_batches(stock.get(name,0), 8)`写了两遍。这样会让代码看起来比较乱，而且如果两个地方忘了同步更新，就会出问题。例如，我们决定每一个批次是4个，而不是8个。如果我们忘了同步修改另一个地方，那么代码可能出问题。

In [22]:
has_bug = {name: get_batches(stock.get(name, 0), 4)
           for name in order
           if get_batches(stock.get(name, 0), 8)}

print('Expected:', found)
print('Found:   ', has_bug)

Expected: {'screws': 4, 'wingnuts': 1}
Found:    {'screws': 8, 'wingnuts': 2}


有个简单的方法可以解决这个问题，那就是在推导的过程中使用Python3.8引入的`:=`海象操作符进行赋值表达。

In [25]:
found = {name: batches for name in order if (batches := get_batches(stock.get(name, 0),8))}
found

{'screws': 4, 'wingnuts': 1}

`batches := get_batches(...)`这条赋值表达式，能从`stock`字典里查找到对应产品一共有几批，并把这个批数放到`batches`变量里。这样，我们推导这个产品所对应批数时，就不用再通过`get_batches`计算了。

在推导过程中，描述新值的那一部分也可以出现赋值表达式。但如果在其他部分引用了定义在那一部分的变量，那么程序可能就会在运行时出错。
例如，如果写成了下面这样，那么程序就必须先考虑`for name, count in stock.items() if tenth > 0`，而这个时候，其中`tenth`还未定义。

In [27]:
result = {name: (tenth := count // 10)
          for name, count in stock.items() if tenth > 0}

NameError: name 'tenth' is not defined

为了解决这个问题，可以把赋值表达式移动到`if`条件里，然后描述新值的这一部分引用已经定义过的`tenth`变量。

In [28]:
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
result

{'nails': 12, 'screws': 3, 'washers': 2}

如果推导逻辑不带条件，而表示新值的那一部分又使用了`:=`操作符，那么操作符左边的变量就会泄漏到包含这条推导语句的那个作用域里。

In [29]:
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

Last item of [62, 17, 4, 12] is 12


这与普通的`for`循环所用的那个循环变量类似。

In [30]:
for count in stock.values():  # Leaks loop variable
    pass
print(f'Last item of {list(stock.values())} is {count}')

Last item of [125, 35, 8, 24] is 24


然而，推导语句中的`for`循环所使用的循环变量，是不会像刚才那样泄漏到外面的。

In [31]:
del count
half = [count // 2 for count in stock.values()]
print(half)   # Works
print(count)  # Exception because loop variable didn't leak

[62, 17, 4, 12]


NameError: name 'count' is not defined

最好不要泄漏循环变量，所以，建议赋值表达式只出现在推导逻辑的条件之中。

赋值表达式不仅可以用在推导过程中，而且可以用来编写生成器表达式。下面这种写法创建的是迭代器，而不是字典实例，该迭代器每次会给出一对数值，其中第一个元素为产品的名字，第二个元素为这种产品的库存。

In [33]:
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))
print(batches) # 还是会跑到生成器表达式外部，注意！！

('screws', 4)
('wingnuts', 1)
1


# #30 不要让函数直接返回列表参数，应该让它逐个生成列表里的值

如果函数要返回的是包含许多结果的序列，那么最简单的办法就是把这些结果放到列表中。例如，我们要返回字符串里每个单词的首字母所对应的下标。

In [34]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

我们把一段文本传给这个函数，可以返回正确的结果。

In [35]:
address = 'Four score and seven years ago...'
result = index_words(address)
result[:10]

[0, 5, 11, 15, 21, 27]

但是该函数有两个缺点。

第一个缺点是，它的代码看起来有点乱。这种函数改用生成器来实现会比较好。生成器由包含`yield`表达式的函数创建。下面就定义一个生成器函数，实现与刚才那个函数相同的效果。

In [37]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

调用生成器函数并不会让其中的代码立刻得到执行，它会返回一个迭代器。把迭代器传给内置的`next`函数，就可以将生成器函数推进到它的下一条`yield`表达式。
生成器会把`yield`表达式的值通过迭代器返回给调用者。

In [38]:
it = index_words_iter(address)
print(next(it))
print(next(it))

0
5


这次的`index_words_iter`函数，比刚才那个函数好懂很多。
如果确实要制作一份列表返回，那可以把生成器函数返回的迭代器传给内置的`list`函数。

In [39]:
result = list(index_words_iter(address))
result[:10]

[0, 5, 11, 15, 21, 27]

`index_words`的第二个缺点是，它必须把所有的结果都保存到列表中。然后才能返回列表。如果输入的数据特别多，那么程序可能会因为内存不足而崩溃。

相反，用生成器函数来实现，就不会有这个问题了。它可以接受长度任意的输入信息，并把内存消耗量压得较低。例如下面这个生成器，只需要把当前这行文字从文件中读进来就行，每次推进的时候，它都只处理一个单词，直到把当前这行文字处理完毕，才读入下一行文字。

In [40]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

该函数运行时所耗的内存，取决于文件中最长的那一行所包含的字符数。

定义这种生成器函数的时候，有一个要注意的地方，就是调用者无法重复使用函数所返回的迭代器，因为这些迭代器是有状态的。

# #31 谨慎地迭代函数所收到的参数

如果函数接受的参数是个包含许多对象的列表，那么这份列表有可能要迭代多次。例如，我们要分析美国得克萨斯州的游客数量。原始数据保存在一份列表中，其中每个元素表示每年有多少游客到这个城市旅游。我们现在要统计每个城市的游客数占游客总数的百分比。

为了求出这份数据，我们编写一个归一化函数`normalzie`，它先把列表里的所有元素加起来求出游客总数，然后，分别用每个城市的游客数除以游客总数计算出该城市在总数中所占的百分比。

In [41]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

把一份范例数据放到列表中传给这个函数，可以得到正确的结果。

In [42]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


为了应对规模更大的数据，我们现在需要让程序能从文件中读取信息，并假设数据都在这份文件中。我们通过生成器实现。

In [45]:
path = 'my_numbers.txt'
with open(path, 'w') as f:
    for i in (15, 35, 80):
        f.write('%d\n' % i)

def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

奇怪的是，对`read_visits`所返回的迭代器调用`normalize`函数后，并没有得到结果。

In [46]:
it = read_visits(path)
percentages = normalize(it)
print(percentages)

[]


出现这种情况的原因在于，迭代器只能产生一个结果。假如迭代器或生成器已经跑出过`StopIteration`异常，继续用它来构造列表或是像`normalize`那样对它做`for`循环，那它不会给出任何结果。

In [47]:
it = read_visits(path)
print(list(it))
print(list(it)) # 已经被使用了

[15, 35, 80]
[]


还有个因素也很令人迷惑，那就是：在已经把数据耗完的迭代器上继续迭代，程序居然不报错。

为了解决这个问题，我们可以把外界传入的迭代器特意遍历一整轮，从而将其中的内容复制到一份列表中。这样的话，就可以用这份列表来处理数据了。

In [48]:
def normalize_copy(numbers):
    numbers_copy = list(numbers)  # Copy the iterator
    total = sum(numbers_copy)
    result = []
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

现在的这个函数就可以正确处理`read_visits`生成器函数所返回的`it`值了。

In [49]:
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这个办法虽然能解决问题，但如果输入的迭代器所要提供的数据量很大。那么就可能导致内存耗尽。
为了应对大规模的数据，其中一个变通方案是让`normalize`函数接受另外一个函数，使它每次要使用迭代器时，都去向那个函数索要。

In [50]:
def normalize_func(get_iter):
    total = sum(get_iter())   # New iterator
    result = []
    for value in get_iter():  # New iterator
        percent = 100 * value / total
        result.append(percent)
    return result

使用`normalize_func`时，需要传入一条`lambda`表达式，让这个表达式去调用`read_visits`生成器函数。这样`normalize_func`每次向`get_iter`索要迭代器时，程序都会给出一个新的迭代器。

In [51]:
percentages = normalize_func(lambda: read_visits(path))
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这样虽然可行，但是传入这么一个`lambda`表达式显得有点生硬。要想用更好的办法解决这个问题，可以新建一种容器类，让它实现迭代器协议。

Python的`for`循环及相关的表达式，正是按照迭代器协议来遍历容器内容的。Python执行`for x in foo`这样的语句时，实际上会调用`iter(foo)`。这个函数会触发`foo.__iter__`的特殊方法，该方法必须返回迭代器对象。最后，Python会用迭代器对象反复调用内置的`next`函数，直到数据耗尽位置(如果抛出`StopIteration`异常，就表示数据已经迭代完了)。

总结起来，只需要让你的类在实现`__iter__`方法时，按照生成器的方式来写就好。

In [58]:
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
    
    def __iter__(self):
        print('Call __iter__')
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

我们只需要把新的容器传给最早的那个`normalzie`函数运行即可，函数本身的代码不需要修改。

In [53]:
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


这样做为什么可以呢？因为`normalize`的`sun`会触发`ReadVisits.__iter__`让系统分配一个新的迭代器对象给它。接下来，`normalize`通过`for`循环计算每项数据占总值的百分比时，又会触发`__iter__`，于是系统又分配另一个迭代器对象。这两个迭代器对象互不影响。这种方案唯一的缺点，就是多次读取输入数据。

明白了`ReadVisits`这种容器的工作原理后，我们就可以在编写函数和方法时先确认一下，收到的应该是像`ReadVisits`这样的容器，而不是普通的迭代器。
如果将普通的迭代器传给内置的`iter`函数，那么函数就会把迭代器本身返回给调用者。反之，如果传来的是容器类型，那么`iter`函数就会返回一个新的迭代器对象。
我们可以借助这条规律，判断输入值是不是这种容器。如果不是，就抛出`TpyeError`异常，因为我们没办法在那样的值上重复遍历。

In [54]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers:  # 说明是迭代器对象
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

还有一种写法也可以检测出这样的问题。`collections.abc`内置模块里定义了名为`Iterator`的类，它用在`isinstance`函数中，可以判断自己收到的参数是不是这种实例。

In [55]:
from collections.abc import Iterator 

def normalize_defensive(numbers):
    if isinstance(numbers, Iterator):  # 如果是，抛出异常
        raise TypeError('Must supply a container')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

这种自定义容器的方案，最适合于既不想像`normalize_copy`函数一样，把迭代器提供的这套数据全部复制一份，同时又要多次遍历这套数据的情况。
该方案不仅支持自定义容器，还支持系统自带的列表类型，因为这些全都属于遵循迭代器协议的可迭代容器。

In [59]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

Call __iter__
Call __iter__


如果输入的是普通迭代器而不是容器，那么`normalize_defensive`就会抛出异常。

In [57]:
visits = [15, 35, 80]
it = iter(visits)
normalize_defensive(it)

TypeError: Must supply a container

# #32 考虑用生成器表达式改写数据量较大的列表推导

列表推导可以根据输入序列中的每个元素创建一个包含派生元素的新列表。如果输入的数据量很大，那么程序就有可能因为内存耗尽而奔溃。

例如，我们要读取一份文件并返回每行的字符数。假如采用列表推导来实现，那需要把文件的每行文本都载入内存，并统计长度。对于庞大的文件，这样写会有问题。下面就是采用列表推导写的代码，只能处理输入数据较少的情况。

In [1]:
import random

with open('my_file.txt', 'w') as f:
    for _ in range(10):
        f.write('a' * random.randint(0, 100))
        f.write('\n')



In [2]:
value = [len(x) for x in open('my_file.txt')]
value

[34, 44, 7, 31, 82, 18, 25, 14, 28, 52]

要想处理大规模的数据，可以使用生成器表达式来做，它扩展了列表推导式与生成器机制。程序在对生成器表达式求值时，并不会让它把包含输出结果的按个序列立刻构建出来，而是会把它当成一个迭代器，该迭代器每次可以根据表达式中的逻辑给出一项结果。

生成器表达式的写法，与列表推导式语句类似，但是它是写在一对圆括号内，而不是方括号内。下面就是生成器表达式的写法，程序不会立刻给出全部结果，而是先将生成器表达式表示成一个迭代器返回。

In [3]:
it = (len(x) for x in open('my_file.txt'))
it

<generator object <genexpr> at 0x7f2d88900970>

返回的迭代器每次可以推进一步，这时它会根据生成表达式的逻辑计算出下一项输出结果(通过内置的`next`函数取得)。需要多少项结果，就把迭代器推进多少次，这种写法不会消耗太多内存。

In [4]:
print(next(it))
print(next(it))

34
44


生成器表达式还有个强大的特性，就是可以组合起来。例如，可以用刚才那条生成器表达式所形成的`it`迭代器作为输入，编写一条新的生成器表达式。

In [5]:
roots = ((x, x**0.5) for x in it)

`roots`迭代器每次推进时，会引发连锁反应：它也推进内部迭代器`it`以判断当前是否还能`it`上面继续迭代，如果可以，就把`it`所返回的值带入`(x, x**0.5)`里求出结果。这种写法同样使用的内存不会太多。

In [6]:
print(next(roots))

(7, 2.6457513110645907)


多个生成器嵌套而成的代码，执行起来还是相当快的。唯一要注意的是，生成器表达式返回的迭代器是有状态的，跑完一轮后，就不能继续使用了。

# #33 通过yield from把多个生成器连起来用

生成器有很多好处，能解决常见的问题。而且可以一个连着一个地用。

例如，我们要编写一个图形程序，让它在屏幕上移动图像，从而形成动画效果。假设要实现这样一段动画：图片先快速移动一段时间，然后暂停，接下来慢速移动一段时间。
为了把移动与暂停表示出来，笔者定义了下面两个生成器函数，让它们分别给出图片在当前时间段内应该保持的速度。

In [7]:
def move(period, speed):
    for _ in range(period):
        yield speed

def pause(delay):
    for _ in range(delay):
        yield 0

为了制作动画，需要将`move`与`pause`连起来用，从而算出这张图片当前的位置与上一个位置之差。
下面的函数用三个`for`循环来表示动画的三个环节，在每个环节里，它都通过`yield`把图片当前的位置与上一次的位置之差`delta`返回给调用者。
根据`animate`函数返回的`delta`值，即可把整段动画做好。

In [8]:
def animate():
    for delta in move(4, 5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2, 3.0):
        yield delta
        

接下来我们就根据`animate`生成器所给出的`delta`值，把整个动画效果渲染出来。

In [9]:
def render(delta):
    print(f'Delta: {delta:.1f}')
    # Move the images onscreen

def run(func):
    for delta in func():
        render(delta)

run(animate)


Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


这种写法的问题在于，`animate`函数里有很多重复的地方。比如它反复地使用`for`结构来操纵生成器，而且每个`for`结构都使用相同的`yield`表达式，这样看上去很啰嗦。本例只连用了三个生成器，就让代码变得如此繁琐，若是动画里面有十几或几十个环节，那么代码读起来会更加困难。

为了解决这个问题，我们可以改用`yield from`形式的表达式来实现。这种形式会先从嵌套进去的小生成器里面取值，如果该生成器已经用完，那么程序的控制流程就会回到`yield from`所在的这个函数之中，然后它有可能进入下一套`yield from`逻辑。

In [10]:
def animate_composed():
    yield from move(4, 5.0)
    yield from pause(3)
    yield from move(2, 3.0)

run(animate_composed)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0


它的运行结果于刚才一样，但是代码看上去更清晰、更直观了。而且这种实现方式要更快。

In [11]:
import timeit

def child():
    for i in range(1_000_000):
        yield i

def slow():
    for i in child():
        yield i

def fast():
    yield from child()

baseline = timeit.timeit(
    stmt='for _ in slow(): pass',
    globals=globals(),
    number=50)
print(f'Manual nesting {baseline:.2f}s')

comparison = timeit.timeit(
    stmt='for _ in fast(): pass',
    globals=globals(),
    number=50)
print(f'Composed nesting {comparison:.2f}s')

reduction = -(comparison - baseline) / baseline
print(f'{reduction:.1%} less time')


Manual nesting 8.10s
Composed nesting 6.16s
24.0% less time


# #34 不要用send给生成器注入数据

`yield`表达式让我们能轻松地写出生成器函数，使得调用者可以每次只获取输出序列中的一项结果。但问题是，这种通道是单向的，即，无法让生成器在其一端接收数据流，同时在另一端给出计算结果。
假如能实现双向通信，那么生成器的使用面会更广。

例如，我们想用软件实现无线广播，用它来发送信号。为了编写这个程序，我们必须用一个函数来模拟正弦波，让它能够给出一系列按照正弦方式分布的点。

In [12]:
import math

def wave(amplitude, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output

有了这个`wave`函数，我们可以让它按照某个固定的振幅生成一系列供传输的值。

In [13]:
def transmit(output):
    if output is None:
        print(f'Output is None')
    else:
        print(f'Output: {output:>5.1f}')

def run(it):
    for output in it:
        transmit(output)

run(wave(3.0, 8))

Output:   0.0
Output:   2.1
Output:   3.0
Output:   2.1
Output:   0.0
Output:  -2.1
Output:  -3.0
Output:  -2.1


这样写可以生成基本的波形，但问题是，该函数在产生这些值的时候，只能按照刚开始给定的振幅来计算，而没办法使振幅在整个过程中根据某个因素发生变化。
现在，我们要让生成器在计算每个值的时候，都能考虑到振幅的变化，从而实现调幅。

Python的生成器支持`send`方法，这可以让生成器变为双向通道。`send`方法可以把参数发给生成器，让它为上一条`yield`表达式的求值结果，并将生成器推进到下一条`yield`表达式，然后把`yield`右边的值返回给`send`方法的调用者。
然而一般情况下，我们还是会通过内置的`next`函数来推进生成器，按照这种写法，上一条`yield`不表达式的求值结果总是`None`。

In [14]:
def my_generator():
    received = yield 1
    print(f'received = {received}')

it = my_generator()
output = next(it)       # 得到第一个生成器的输出
print(f'output = {output}')

try:
    next(it)            # 推进生成器直到退出
except StopIteration:
    pass
else:
    assert False

output = 1
received = None


如果不通过`for`循环或内置的`next`函数推进生成器，而是改用`send`方法，那么调用方法时传入的参数就会成为上一条`yield`表达式的值，生成器拿到这个值后，会继续运行到下一条`yield`表达式那里。可是，刚开始推进生成器的时候，它是从头执行的，而不是从某一条`yield`表达式那里继续的，所以，首次调用`send`方法时，只能传`None`，要是传入其他值，程序运行时就会抛出异常。

In [15]:
it = iter(my_generator())
output = it.send(None) # 得到第一个生成器的输出
print(f'output={output}')
try:
    it.send('Hello!')
except StopIteration:
    pass

output=1
received = Hello!


我们可以利用这种机制让调用者把振幅发送过来，这样函数就能根据这个输入值调整生成的正弦波幅值了。 
首先修改`wave`函数的代码，让它把`yield`表达式的求值结果保存到`amplitude`变量里，这样就能根据该变量计算出下次应该生成的值。

In [16]:
def wave_modulating(steps):
    step_size = 2 * math.pi / steps
    amplitude = yield             # 接收初始的振幅
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        amplitude = yield output  # 接收下一个振幅

然后，要修改`run`函数调用`wave_modulating`函数的方式。它现在必须把每次所要使用的振幅发给`wave_modulating`生成器。首次必须发送`None`。

In [17]:
def run_modulating(it):
    amplitudes = [
        None, 7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    for amplitude in amplitudes:
        output = it.send(amplitude)
        transmit(output)

run_modulating(wave_modulating(12))

Output is None
Output:   0.0
Output:   3.5
Output:   6.1
Output:   2.0
Output:   1.7
Output:   1.0
Output:   0.0
Output:  -5.0
Output:  -8.7
Output: -10.0
Output:  -8.7
Output:  -5.0


这样写没问题，程序可以按照每次输入的值调整输出信号的振幅。
但是这种写法有个缺点，就是他很难让初次阅读代码的人立刻理解它的意思。把`yield`放在赋值语句的右侧，本身就不太直观。
现在假设程序的需求变得更加复杂了。这次要生成的不是简单的正弦波，而是由多个信号序列构成的复合波形。要实现这个需求，可以连用几条`yield from`语句，把多个生成器串接起来。下面先验证一下。

In [18]:
def complex_wave():
    yield from wave(7.0, 3)
    yield from wave(2.0, 4)
    yield from wave(10.0, 5)

run(complex_wave())

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5


可以看到，`yield from`表达式确实可以处理比较简单的情况。我们再看看它能否处理那种采用`send`方法写成的函数。下面就用这种写法，接连多次调用`wave_modulating`生成器。

In [19]:
def complex_wave_modulating():
    yield from wave_modulating(3)
    yield from wave_modulating(4)
    yield from wave_modulating(5)

run_modulating(complex_wave_modulating())

Output is None
Output:   0.0
Output:   6.1
Output:  -6.1
Output is None
Output:   0.0
Output:   2.0
Output:   0.0
Output: -10.0
Output is None
Output:   0.0
Output:   9.5
Output:   5.9


出现了很多`None`。因为每条`yield from`表达式其实都在遍历一个嵌套进去的生成器，所以每个嵌套生成其都必须分别执行它们各自的第一条`yield`语句(也就是什么值都不带的`yield`语句)，只有执行过这条语句之后，这些生成器才能通过`send`方法所传来的值决定这条语句的求值结果，并把这个结果放在`amplitude`变量里以计算下一次应该输出的值。

所以`complext_wave_modulating`函数处理完前一个嵌套的生成器之后，会进入下一个嵌套的生成器，而这是就必须先把该生成器的第一条`yield`语句运行过去，这就导致后面两个嵌套生成器会各自从`amlitudes`列表里浪费掉一个值，并使得每个嵌套生成器所拿到的第一个结果必定是`None`，还会让最后那个嵌套生成器少执行两次。

也就是说，`yield from`语句和`send`方法结合使用效果不太让人满意。
最简单的一种写法，是把迭代器传给`wave`函数，让`wave`每次用到振幅的时候，通过Python内置的`next`函数推进这个迭代器并返回一个输入振幅。

In [20]:
def wave_cascading(amplitude_it, steps):
    step_size = 2 * math.pi / steps
    for step in range(steps):
        radians = step * step_size
        fraction = math.sin(radians)
        amplitude = next(amplitude_it)  # 得到下一个输入
        output = amplitude * fraction
        yield output


这样，我们只需要把同一个迭代器分别传给几条`yield from`语句里的`wave_casading`就行。迭代器是有状态的，所以下一个`wave_cascading`会从上一个使用完的地方，继续往下使用`amplitude_it`迭代器。

In [21]:
def complex_wave_cascading(amplitude_it):
    yield from wave_cascading(amplitude_it, 3)
    yield from wave_cascading(amplitude_it, 4)
    yield from wave_cascading(amplitude_it, 5)

要想触发这个组合的迭代器，只需要把振值放在列表汇总，并把针对列表制作的迭代器传给`complex_wave_cascading`就好。

In [22]:
def run_cascading():
    amplitudes = [7, 7, 7, 2, 2, 2, 2, 10, 10, 10, 10, 10]
    it = complex_wave_cascading(iter(amplitudes))
    for amplitude in amplitudes:
        output = next(it)
        transmit(output)

run_cascading()

Output:   0.0
Output:   6.1
Output:  -6.1
Output:   0.0
Output:   2.0
Output:   0.0
Output:  -2.0
Output:   0.0
Output:   9.5
Output:   5.9
Output:  -5.9
Output:  -9.5


这种写法最大的优点在于，迭代器可以来自任何地方，而且完全可以是动态的。此方案只有一个缺陷，就是必须假设入则输入的生成器绝对能保证线程安全。

# #35 不要通过throw变换生成器的状态

除`yield from`表达式与`send`方法外，生成器还有一项高级功能，就是可以把调用者通过`throw`方法传来的`Exception`实例重新抛出。
如果调用了`throw`方法，那么生成器下次推进时，就不会像平常那样，直接走到下一条`yield`表达式那里，而是会把通过`throw`方法传入的异常重新抛出。

In [23]:
class MyError(Exception):
        pass

def my_generator():
    yield 1
    yield 2
    yield 3

it = my_generator()
print(next(it))  # Yield 1
print(next(it))  # Yield 2
print(it.throw(MyError('test error')))

1
2


MyError: test error

生成器函数可以用标准的`try/except`复合语句把`yield`表达式包裹起来，如果函数上次执行到了这条表达式这里，这次即将继续执行时，又发现外界通过`throw`方法给自己注入了异常，那么这个异常就会被`try`结构捕获下来，如果捕获之后不继续抛出异常，那么生成器函数会推进到下一条`yield`表达式。

In [24]:
def my_generator():
    yield 1

    try:
        yield 2
    except MyError:
        print('Got MyError!')
    else:
        yield 3

    yield 4

it = my_generator()
print(next(it))  # Yield 1
print(next(it))  # Yield 2
print(it.throw(MyError('test error')))

1
2
Got MyError!
4


这项机制会在生成器与调用者之间形成双向通信通道，这在某些情况下是有用的。
例如，要编写一个偶尔可以重置的计时器程序。笔者定义下面的`Reset`异常与`timer`生成器方法，让调用者可以在`timer`给出的迭代器上通过`throw`方法注入`Reset`异常，令计时器重置。

In [25]:
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

这个计时器可以与外界某个按秒轮询的输入机制对接起来。为此，笔者定义一个`run`函数以驱动`timer`生成器所给出的那个`it`迭代器，并根据外界的情况做处理，如果外界要求重置，那就通过`it`迭代器的`throw`方法给计时器注入`Reset`变量，如果没有这种要求，那就调用`announce`函数打印生成器所给的倒计时值。

In [26]:
RESETS = [
    False, False, False, True, False, True, False,
    False, False, False, False, False, False, False]

def check_for_reset():
    # Poll for external event
    return RESETS.pop(0)

def announce(remaining):
    print(f'{remaining} ticks remaining')

def run():
    it = timer(4)    
    while True:
        try:
            if check_for_reset():
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


这样写没错，但是有点难懂。
有个简单的办法，能改写这段代码，那就是用可迭代的容器对象定义一个有状态的闭包。

In [27]:
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period

    def reset(self):
        self.current = self.period

    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current


现在， `run`函数就好写多了，这样写出来的代码没有那么多层嵌套，读起来很容易。

In [28]:
RESETS = [
    False, False, True, False, True, False,
    False, False, False, False, False, False, False]

def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset():
            timer.reset()
        announce(current)

run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


凡是想用生成器与异常来实现的功能，通常都可以改用异步机制去做，那样会更好。如果确实遇到了这里将的这种需求，那么应该通过可迭代的类来实现生成器，而不要用`throw`方法注入异常。

# #36 考虑用itertools拼装迭代器与生成器

Python内置的`itertools`模块里面有很多函数，可以用来安排迭代器之间的交互关系。

## 连接多个迭代器

**chain**
`chain`可以把多个迭代器从头到尾连成一个迭代器。

In [29]:
import itertools

it = itertools.chain([1,2,3],[4,5,6])
list(it)

[1, 2, 3, 4, 5, 6]

**repeat**
`repeat`可以制作这样一个迭代器，它会不停地输出某个值。它的第二个参数指定迭代器最多能输出几次。

In [30]:
it = itertools.repeat('hello', 3)
list(it)

['hello', 'hello', 'hello']

**cycle**
`cycle`可以制作这样一个迭代器，它会循环地输出某段内容之中的各项元素。

In [31]:
it = itertools.cycle([1, 2])
result = [next(it) for _ in range (10)]
print(result)

[1, 2, 1, 2, 1, 2, 1, 2, 1, 2]


**tee**
`tee`可以让一个迭代器分裂成多个平行的迭代器，具体个数由第二个参数指定。如果这些迭代器推进的速度不一致，那么程序可能需要大量内存做缓存，以存放进度落后的迭代器将来用到的元素。

In [32]:
it1, it2, it3 = itertools.tee(['first', 'second'], 3)
print(list(it1))
print(list(it2))
print(list(it3))

['first', 'second']
['first', 'second']
['first', 'second']


**zip_longest**
它与Python内置的`zip`函数类似，区别在于，如果源迭代器的长度不同，那么他会用`fillvalue`的值来填补提前耗尽的那些迭代器所留下的空缺。

In [33]:
keys = ['one', 'two', 'three']
values = [1, 2]

normal = list(zip(keys, values))
print('zip:        ', normal)

it = itertools.zip_longest(keys, values, fillvalue='nope')
longest = list(it)
print('zip_longest:', longest)

zip:         [('one', 1), ('two', 2)]
zip_longest: [('one', 1), ('two', 2), ('three', 'nope')]


## 过滤源迭代器中的元素

Python内置的`itertools`模块有一些函数可以过滤源迭代器中的元素。

**islice**
`islice`可以在不拷贝数据的前提下，按照下标切割源迭代器。可以只给出切割的终点，也可以同时给出起点和终点，还可以指定步进值。

In [34]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

first_five = itertools.islice(values, 5)
print('First five: ', list(first_five))

middle_odds = itertools.islice(values, 2, 8, 2)
print('Middle odds:', list(middle_odds))

First five:  [1, 2, 3, 4, 5]
Middle odds: [3, 5, 7]


**takewhile**
`takewhile`会一直从源迭代器里获取元素，直到某元素让测试函数返回`False`为止。

In [35]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
less_than_seven = lambda x: x < 7
it = itertools.takewhile(less_than_seven, values)
print(list(it))

[1, 2, 3, 4, 5, 6]


**dropwhile**
与`takewhile`相反，`dropwhile`会一直跳过源序列里的元素，直到某元素让测试函数返回`True`为止，然后它会从这个地方逐个取值。

In [36]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
less_than_seven = lambda x: x < 7
it = itertools.dropwhile(less_than_seven, values)
print(list(it))

[7, 8, 9, 10]


**filterfalse**
`filterfalse`和内置的`filter`函数相反，它会逐个输出源迭代器里使得测试函数返回`False`的那些元素。

In [37]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = lambda x: x % 2 == 0

filter_result = filter(evens, values)
print('Filter:      ', list(filter_result))

filter_false_result = itertools.filterfalse(evens, values)
print('Filter false:', list(filter_false_result))

Filter:       [2, 4, 6, 8, 10]
Filter false: [1, 3, 5, 7, 9]


## 用源迭代器中的元素合成新元素

**accumulate**

`accumulate`会从源迭代器里取出一个元素，并把已经累计的结果与这个元素一起传给表示累加逻辑的函数，然后输出那个函数的计算结果，并把结果当成新的累计值。

In [38]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
sum_reduce = itertools.accumulate(values)
print('Sum:   ', list(sum_reduce))

def sum_modulo_20(first, second):
    output = first + second
    return output % 20

modulo_reduce = itertools.accumulate(values, sum_modulo_20)
print('Modulo:', list(modulo_reduce))

Sum:    [1, 3, 6, 10, 15, 21, 28, 36, 45, 55]
Modulo: [1, 3, 6, 10, 15, 1, 8, 16, 5, 15]


这与内置的`functools`模块中的`reduce`函数实际上是一样的，只不过这个函数每次只给出一项累计值。默认的逻辑是两值相加。

**product**

`product`会从一个或多个源迭代器里获取元素，并计算笛卡尔积，它可以取代那种多层嵌套的列表推导代码。

In [39]:
single = itertools.product([1, 2], repeat=2)
print('Single:  ', list(single))

multiple = itertools.product([1, 2], ['a', 'b'])
print('Multiple:', list(multiple))

Single:   [(1, 1), (1, 2), (2, 1), (2, 2)]
Multiple: [(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


**permutations**

`permutations`会考虑源迭代器所能给出的全部元素，并逐个输出由其中N个元素形成的每种有序排列方式，元素相同但顺序不同，算作两种排列。

In [47]:
from pprint import pprint

it = itertools.permutations([1, 2, 3, 4], 2)
pprint(list(it))

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3)]


**combinations**

`combinations`会考虑迭代器所能给出的全部元素，并逐个输出由其中N个元素形成的每种无序组合方式，元素相同但顺序不同，算作同一种组合。

In [48]:
it = itertools.combinations([1, 2, 3, 4], 2)
pprint(list(it))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]


**combinations_with_replacement**
`combinations_with_replacement`与`combinations`类似，但它允许同一个元素在组合里多次出现。

In [49]:
it = itertools.combinations_with_replacement([1, 2, 3, 4], 2)
pprint(list(it))

[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]
